In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

We'll begin camera calibration with loading given images. 

In [ ]:
import glob
def load_images(load_path):
    image_paths = glob.glob(load_path)
    images = [cv2.imread(path) for path in image_paths]
    return images
        
def display_grid(images, n_col, title):
    plt.close('all')
    fig, ax_arr = plt.subplots(len(images)//n_col, n_col,figsize=(15,7))
    fig.suptitle(title, fontsize=30)
    for i, image in enumerate(images):
        ax = ax_arr[i // n_col, i % n_col]
        ax.imshow(image)
        ax.set_xticks([]), ax.set_yticks([])
    plt.show()


calibration_images = load_images("camera_cal/*.jpg")
display_grid(calibration_images, 5, "Images for camera calibration")

As shown above there are 20 images. Using them we could do camera calibration.



In [ ]:
def find_corners(image, pattern_size):
    """
    Look for the corners on chessboard image given pattern size
    
    Args:
        image: cv2.image
        pattern_size: tuple
    
    Returns:
        List of corners coordinates if they are found
    """
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    retval, corners = cv2.findChessboardCorners(gray, pattern_size)
    return corners if retval else []

import functools

def find_calibration_parameters(images, pattern_size):
    return_corners = list(map(lambda x: find_corners(x, pattern_size), images))
    corners = list(filter(lambda x: len(x) != 0, return_corners))

    objp_size = np.prod(pattern_size), 3
    objp = np.zeros(objp_size, np.float32)
    objp[:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2)
    objpoints = [objp] * len(corners)
    
    image_size = images[-1].shape[1::-1]
    ret, mtx, dist, *rest = cv2.calibrateCamera(objpoints, corners, image_size, None, None)
    
    return ret, mtx, dist, return_corners

def undistort_images(images, mtx, dist):
    return [cv2.undistort(image, mtx, dist) for image in images]

def transform_perspective(image, corners, pattern_size):
    if len(corners) == 0:
        img = image.copy()
        cv2.putText(img, "Failed", (10, 100), cv2.FONT_ITALIC, 4.0, (0, 0, 255), 3)
        return img
    nx, ny = pattern_size
    src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
    image_size = image.shape[1::-1]
    w, h = image_size[0], image_size[1]
    x, y = (w / (nx - 1), h / (ny - 1))
    d = (x + y) / 2.4
    dst = np.float32([[d, d], [w - d, d], [w-d, h-d], [d, h - d]])

    M = cv2.getPerspectiveTransform(src, dst)
    return cv2.warpPerspective(image, M, (w, h))

ps = (9, 6)
ret, mtx, dist, corners = find_calibration_parameters(calibration_images, pattern_size=ps)

if ret:
    undistorted_images = undistort_images(calibration_images, mtx, dist)
    display_grid(undistorted_images, 5, "Undistorted images")

    transformed_images = [transform_perspective(image, corners[i], pattern_size=ps) \
                          for i, image in enumerate(undistorted_images)]
    display_grid(transformed_images, 5, "Transformed images")
